This notebook is for converting a TF Saved Graph to TF Frozen Graph and then to TRT

## Setup

In [15]:
%%bash

echo 'Train:'
TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f
echo
echo 'Convert (Saved Graph):'
CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort
echo
echo 'Convert (Frozen Graph):'
CONVERT_TF_FROZEN_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/frozen
find ${CONVERT_TF_FROZEN_GRAPH_DIR} -type f | sort
echo
echo 'Convert TRT:'
CONVERT_TRT_DIR=../../../tmp/trt_end_to_end/convert/trt
find ${CONVERT_TRT_DIR} -type f | sort

Train:
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.h5
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.md
../../../tmp/trt_end_to_end/train/conv/001/params.json
../../../tmp/trt_end_to_end/train/basic/001/params.json
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.md
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.h5
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.md
../../../tmp/trt_end_to_end/train/batchn/001/params.json
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.h5
../../../tmp/trt_end_to_end/train/resnet50/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.md

Convert (Saved Graph):
../../../tmp/trt_end_to_end/convert/tf/saved/basic/001/checkpoint
../../../tmp/trt_end_to_end/conv

In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
_NAME = 'resnet50'
_EPOCH = 1

In [4]:
from src.utils.trt_end_to_end_constants import *
_NAME, _EPOCH, _TIME = get_params(_NAME, _EPOCH)

## Import

In [5]:
# import the needed libraries
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
print(tf.__name__, tf.__version__, sep='-')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

tensorflow-1.14.0


In [ ]:
from src.utils.trt_end_to_end_constants import *

md_filename = MD_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
_train_dir = get_train_dir(_NAME, _EPOCH)
md_filepath = os.path.join(_train_dir, md_filename)
print(md_filepath)

In [7]:
from src.meta.metadata import Metadata
ret, metadata = Metadata.from_md(md_filepath)
assert ret == 0

from pprint import pprint
pprint(vars(metadata))

{'epoch': 1,
 'input_names': ['resnet50_input:0'],
 'name': 'resnet50',
 'output_names': ['fc100/Softmax:0']}


## Freeze Graph

Input: Saved Graph

In [8]:
saved_graph_dir = get_saved_graph_dir(_NAME, _EPOCH)

saver_filename = SAVED_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
saver_filepath = os.path.join(saved_graph_dir, saver_filename)
print(saver_filepath)

meta_filename = SAVED_GRAPH_META_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
meta_filepath = os.path.join(saved_graph_dir, meta_filename)
print(meta_filepath)

../../../tmp/trt_end_to_end/convert/tf/saved/resnet50/001/tf_model_resnet50_epoch001_2019-09-03T19:31
../../../tmp/trt_end_to_end/convert/tf/saved/resnet50/001/tf_model_resnet50_epoch001_2019-09-03T19:31.meta


Output: Frozen Graph

In [9]:
frozen_graph_dir = get_frozen_graph_dir(_NAME, _EPOCH)

frozen_graph_filename = FROZEN_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
frozen_graph_filepath = os.path.join(frozen_graph_dir, frozen_graph_filename)
print(frozen_graph_filepath)

../../../tmp/trt_end_to_end/convert/tf/frozen/resnet50/001/resnet50_epoch001_2019-09-03T19:31_frozen.pb


In [10]:
# has to be use this setting to make a session for TensorRT optimization
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
    # import the meta graph of the tensorflow model
    saver = tf.train.import_meta_graph(meta_filepath)
    # then, restore the weights to the meta graph
    saver.restore(sess, saver_filepath)
    
    # specify which tensor output you want to obtain 
    # (correspond to prediction result)
    
    #your_outputs = ["basic2/Softmax"]
    your_outputs = [output.split(':')[0] for output in metadata.output_names]
    
    # convert to frozen model
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, # session
        tf.get_default_graph().as_graph_def(),# graph+weight from the session
        output_node_names=your_outputs)
    #write the TensorRT model to be used later for inference
    with gfile.FastGFile(frozen_graph_filepath, 'wb') as f:
        f.write(frozen_graph.SerializeToString())
    print("Frozen model is successfully stored to %s" % frozen_graph_filepath)

W0903 19:49:35.573256 140174754711360 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0903 19:49:36.889784 140174754711360 deprecation.py:323] From <ipython-input-10-41cb528a4cc4>:18: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0903 19:49:36.890823 140174754711360 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.com

Frozen model is successfully stored to ../../../tmp/trt_end_to_end/convert/tf/frozen/resnet50/001/resnet50_epoch001_2019-09-03T19:31_frozen.pb


## Convert to TRT
Input: Frozen Graph

Output: TRT Graph

In [11]:
trt_graph_dir = get_trt_graph_dir(_NAME, _EPOCH)
trt_graph_filename = TRT_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
trt_graph_filepath = os.path.join(trt_graph_dir, trt_graph_filename)
print(trt_graph_filepath)

../../../tmp/trt_end_to_end/convert/trt/resnet50/001/resnet50_epoch001_2019-09-03T19:31_trt.pb


In [12]:
# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=your_outputs,
    max_batch_size=2,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile(trt_graph_filepath, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored to %s" % trt_graph_filepath)

TensorRT model is successfully stored to ../../../tmp/trt_end_to_end/convert/trt/resnet50/001/resnet50_epoch001_2019-09-03T19:31_trt.pb


In [13]:
%%bash

CONVERT_TRT_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/trt
find ${CONVERT_TRT_GRAPH_DIR} -type f | sort

../../../tmp/trt_end_to_end/convert/trt/basic/001/basic_epoch001_2019-09-03T19:15_trt.pb
../../../tmp/trt_end_to_end/convert/trt/batchn/001/batchn_epoch001_2019-09-03T19:28_trt.pb
../../../tmp/trt_end_to_end/convert/trt/conv/001/conv_epoch001_2019-09-03T19:30_trt.pb
../../../tmp/trt_end_to_end/convert/trt/resnet50/001/resnet50_epoch001_2019-09-03T19:31_trt.pb


## Analyze trt_engine_nodes

In [14]:
# check how many ops of the original frozen model
all_nodes = len([1 for n in frozen_graph.node])
print("numb. of all_nodes in frozen graph:", all_nodes)

# check how many ops that is converted to TensorRT engine
trt_engine_nodes = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("numb. of trt_engine_nodes in TensorRT graph:", trt_engine_nodes)
all_nodes = len([1 for n in trt_graph.node])
print("numb. of all_nodes in TensorRT graph:", all_nodes)

numb. of all_nodes in frozen graph: 875
numb. of trt_engine_nodes in TensorRT graph: 1
numb. of all_nodes in TensorRT graph: 3
